# Avanced 21-cm Cosmology
## [Shikhar Mittal (TIFR)](https://sites.google.com/view/shikharmittal/home)
shikhar.mittal@tifr.res.in

If you use this notebook please cite [Mittal & Kulkarni (2022)](https://doi.org/10.1093/mnras/stac1961).

**Goal**: In this notebook we will calculate:
1. the luminosity function
2. number of galaxies seen by a survey
3. compare our results with *HST* and *JWST* data

**Theory**: Luminosity for a given halo mass $M_{\mathrm{h}}$ is
$$L_\mathrm{UV}=f_\star\dot{M}_{0}\left(\frac{M_{\mathrm{h}}}{10^{10}\mathrm{M}_{\odot}}\right)^a\left(\frac{1+z}{7}\right)^b\mathcal{L}_{\mathrm{UV}}\,,$$
where $\mathcal{L}_{\mathrm{UV}}=8.695\times10^{20}\,\mathrm{WHz^{-1}(\mathrm{M}_{\odot}yr^{-1})^{-1}}$, $\dot{M}_{0}=3\,\mathrm{M}_{\odot}$yr$^{-1}$, $a=1.127$ and $b=2.5$.

Absolute magnitude is [(Oke 1974)](http://dx.doi.org/10.1086/190287)

$$
M_\mathrm{UV}=-2.5\log_{10}\left[\frac{1}{4\pi}\left(\frac{L_\mathrm{UV}}{1\,\mathrm{WHz^{-1}}}\right)\left(\frac{ d_{10}}{1\,\mathrm{m}}\right)^{-2}\right]-56.1\,,
$$
where $d_{10}=10\,$pc.

The luminosity function is given by (called the halo abundance matching technique)
$$\mathrm{LF}=\frac{d \varphi(M_{\lambda})}{d M_{\lambda}}=\frac{d n(M_{\mathrm{h}})}{d M_{\mathrm{h}}}\frac{d M_{\mathrm{h}}}{d L_\lambda}\frac{d L_\lambda}{d M_\lambda}\,,$$
where $d n$ is comoving number density of haloes with mass between $M_{\mathrm{h}}$ and $M_{\mathrm{h}}+d M_{\mathrm{h}}$, $L_\lambda$ is the specific luminosity at a wavelength $\lambda$, and $M_\lambda$ is the corresponding absolute magnitude.

For $f_\star$ independent of $M_{\mathrm{h}}$ we get
$$
\mathrm{LF}=-\frac{2\ln10}{5a}\frac{d n}{d \ln M_\mathrm{h}}\,,
$$
Eliminate $M_{\mathrm{h}}$ from these equations to get LF as a function of $M_\mathrm{UV}$.

The number of galaxies brighter than a given limiting magnitude 
$$
N(m_{\mathrm{UV,lim}}\ ,z)=\Delta z\frac{c}{H(z)}\left(\frac{d_\mathrm{L}}{1+z}\right)^2\int_{M_{\mathrm{h,lim}}}^{\infty}\ \frac{d n}{d M_{\mathrm{h}}}d M_{\mathrm{h}}\,,
$$
where $\Delta z=1$ is the redshift bin, $M_{\mathrm{h,lim}}$ is the limiting halo mass corresponding to limiting apparent magnitude $m_{\mathrm{UV,lim}}\,$ and $d_\mathrm{L}$ is the luminosity distance.

I have chosen the default value of $f_\star=0.1$.

First load all the packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as scsp
import scipy.integrate as scint
from colossus.cosmology import cosmology
from colossus.lss import mass_function
import sys

Define all the constants and parameters

In [ ]:
#Universal constants
GN=6.67e-11 #Gravitational constant
cE=2.998e8  #Speed of light
kB=1.38e-23 #Boltzmann constant
hP=6.634e-34 #Planck's contant
mP=1.67e-27 #Mass of proton
me=9.1e-31 #Mass of electron
eC=1.6e-19 #Charge of electron
epsilon=8.85e-12 #Permittivity of free space

aS=7.52e-16 #Stephan's constant
sigT=6.65e-29 #Thomson scattering cross-section, m^2
#-------------------------------------------------------------
#Cosmology
my_cosmo = {'flat': True, 'H0': 67.4, 'Om0': 0.315, 'Ob0': 0.049, 'sigma8': 0.811, 'ns': 0.965,'relspecies': False,'Tcmb0': 2.725}
cosmo = cosmology.setCosmology('my_cosmo', my_cosmo)

Mpc2km = 3.0857e19
Ho = my_cosmo['H0'] #Hubble parameter today; km/s/Mpc
h100 = Ho/100 #Hubble parameter today; 1 km/s/Mpc
Om_m = my_cosmo['Om0']
Om_b = my_cosmo['Ob0']
Om_lam = 1-Om_m
Tcmbo = my_cosmo['Tcmb0']
Yp = 0.245 #primordial helium fraction
fnu = 0.68 #neutrino contribution to energy density in relativistic species; 3 massless nu's 
rho_crit=3*Ho**2/(8*np.pi*GN*Mpc2km**2) #critical density of the Universe today; kgm^-3
Om_r = (1+fnu)*aS*Tcmbo**4/(cE**2*rho_crit)

Set your free parameters here.

In [ ]:
a=1.127
b=2.5
Mdot0=3     #Solar mass per year
l=8.695e20  #W/Hz/(Msun/yr)

Define all the functions.

In [ ]:
def plotter(x,y,xlog=False,ylog=True,quant_name='',par_name='',params=[],add_hst=False,add_jwst=False):
    plt.rc('text', usetex=True)
    plt.rc('font', family='serif')

    fig,ax=plt.subplots(figsize=(8.3,7.5),dpi=300)
    fig.subplots_adjust(left=0.12, bottom=0.07, right=0.88, top=0.97)
    clr=['b','r','limegreen','magenta','dodgerblue']
    
    heading=''
    if par_name=='z':
        heading = r'$z$'
    elif par_name=='survey':
        heading = 'Survey'
    elif par_name=='fs' or par_name=='fstar':
        heading = r'$f_{\star}$'
        ax.invert_xaxis()
    else:
        print('Warning: unknown parameter')

    if np.ndim(y)>1:
        leng = np.ndim(y)
        for i in range(leng):
            ax.plot(x,y[i,:],color=clr[i],label=params[i])
        ax.legend(title=heading,title_fontsize=18,fontsize=18,frameon=False)
    elif type(y)==np.ndarray:
        ax.plot(x,y,'b',label='Our prediction at ' + heading + '$\,=\,${}'.format(params))
        ax.legend(fontsize=18,frameon=False)
    else:
        print("Error: incorrect syntax! Give y as an array or a list of arrays. eg. [y1, y2, y3]")
        sys.exit()

    if xlog==True:
        ax.set_xscale('log')
    if ylog==True:
        ax.set_yscale('log')

    if quant_name=='hmf' or quant_name=='HMF':
        ax.set_ylabel(r'$\frac{\mathrm{d} n}{\mathrm{d}\ln M_\mathrm{h}}\,\mathrm{cMpc^{-3}}$',fontsize=20)
        ax.set_xlabel(r'$M_{\mathrm{h}}\,(\mathrm{M}_\odot)$',fontsize=20)
    elif quant_name=='lf' or quant_name=='LF':
        if add_hst==True:
            MUV_hst=np.array([-22.52,-22.02,-21.52 ,-21.02 ,-20.52 ,-20.02 ,-19.52 ,-18.77 ,-17.77 ,-16.77])
            LF_hst=np.array([0.000002,0.000015,0.000053,0.000176,0.000320,0.000698,0.001246,0.001900,0.006680,0.013640])
            err_hst=np.array([0.000002,0.000006,0.000012,0.000025,0.000041,0.000083, 0.000137,0.00032,0.00138,0.0042])
            ax.errorbar(MUV_hst,LF_hst,err_hst,color='k',lw=1.5, ls=':', label=r'HST $(z\approx6)$')
            ax.legend(fontsize=18,frameon=False)
        elif add_jwst==True:
            MUV_jwst=np.array([-22.1037, -20.8395, -20.5819, -19.7625, -19.2709])
            LF_jwst=10**np.array([-6.09343, -5.02768, -4.76125, -3.98616, -3.84083])
            ax.scatter(MUV_jwst,LF_jwst,color='k',lw=1.5, ls=':', label=r'JWST $(z\approx11)$')
            ax.legend(fontsize=18,frameon=False)
        
        ax.set_ylabel(r'$\left|\mathrm{d}\varphi/\mathrm{d}M_{\mathrm{UV}}\right|\bigg(\mathrm{cMpc}^{-3}\,\mathrm{mag}^{-1}\bigg)$',fontsize=20)
        ax.set_xlabel(r'$M_{\mathrm{UV}}$',fontsize=20)
    elif quant_name=='n' or quant_name=='N':
        ax.set_ylabel(r'$N$',fontsize=20)
        ax.set_xlabel(r'$1+z$',fontsize=20)
    else:
        print("Warning: unknown quantity. Plotting anyway!")

    
    ax.tick_params(axis='both', which='major', length=5, width=1, labelsize=20,direction='in',pad=10)
    ax.tick_params(axis='both', which='minor', length=3, width=1, direction='in')
    ax.minorticks_on()
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    ax.set_aspect(1.0/ax.get_data_ratio(), adjustable='box')
    plt.show()
    #plt.savefig(quant_name+'.pdf')
    return 

In [ ]:
def H(z):  #Hubble factor in SI units. sec^-1
    return Ho*(Om_r*(1+z)**4+Om_m*(1+z)**3+Om_lam)**0.5/Mpc2km

def hmf(Mh,z):#Mh in the halo mass in units of solar mass; output in cMpc**-3
    M=Mh*h100 #M in units of solar mass/h
    return h100**3*mass_function.massFunction(M, z, q_in='M', q_out='dndlnM', model = 'press74')

def dndM(Mh,z):#Mh in the halo mass in units of solar mass; output in cMpc**-3.Msun**-1
    M=Mh*h100 #M in units of solar mass/h
    return h100**3*1/Mh*mass_function.massFunction(M, z, q_in='M', q_out='dndlnM', model = 'press74')
    
def luminosity(Mh,z,fstar): #For given halo mass (in solar mass) and redshift, what is the luminosity in W/Hz
    return fstar*Mdot0*(Mh/1e10)**a*((1+z)/7)**b*l

def MAB(Mh,z,fstar=0.1):  #Absolute AB magnitude, Mh in solar mass units.
    return -2.5*np.log10(luminosity(Mh,z,fstar)/(4*np.pi*(10*3.086e16)**2))-56.1

#For given halo mass (in solar mass) and redshift, get LF. It is in same units as HMF.
#Also, note that this model is valid when SFE is a constant.
def lum_func(Mh,z):
    return 2*np.log(10)/5/a*hmf(Mh,z)

def muv2Mh(muv,z,fstar=0.1): #muv is apparent magnitude; output is halo mass in solar mass units 
    DL = cosmo.luminosityDistance(z)/h100*3.086e22 #luminosity distance in meters.
    Lum = 4*np.pi*DL**2*10**(-0.4*(muv+56.1)) #Halo luminosity in W/Hz
    return 1e10*(Lum/(fstar*Mdot0*((1+z)/7)**b*l))**(1/a)

#Given a limiting apparent magnitude and survey area (in deg), what is the number of galaxies seen at z
def num_gal(muv_lim,area,z,fstar=0.1):
    def Ngal(muv_lim,area,z,fstar):
        Mh_lim = muv2Mh(muv_lim,z,fstar)
        halo_masses=np.logspace(np.log10(Mh_lim),18,2000)
        integral=scint.trapz(dndM(halo_masses,z),halo_masses)    #number per unit cMpc^3
        DL = cosmo.luminosityDistance(z)/h100 #luminosity distance in Mpc
        return 1/3.086e22*cE/H(z)*(DL/(1+z))**2*(np.pi/180)**2*integral*area
    
    if type(z)==np.ndarray:
        leng = len(z)
        N=np.zeros(leng)
        count=0
        for i in z:
            N[count]=Ngal(muv_lim,area,i,fstar)
            count=count+1
    elif type(z)==list:
        leng = len(z)
        N=np.zeros(leng)
        count=0
        for i in z:
            N[count]=Ngal(muv_lim,area,i,fstar)
            count=count+1
    else:
        N=Ngal(muv_lim,area,z,fstar)
        print('For survey area =',area,'deg and limiting magnitude =',muv_lim,'there are',round(N),'galaxies at z =',z)
    return N

In [ ]:
Mh=np.logspace(10,12) #range of halo masses (in solar mass). For JWST choose 10-11.5. For HST choose 10-12
z=6  #and a redshift. For JWST choose z=9.2 and for HST choose z=6.

#compute the luminosity function and the corresponding absolute magnitude.
LF = lum_func(Mh,z)
Muv = MAB(Mh,z)

#You want to plot LF vs Muv, because that is how observers report their result

In [ ]:
#Before jumping into LFs, try plotting the halo mass function
plotter(Mh,hmf(Mh,z=6),xlog=True,ylog=True,quant_name='hmf',par_name='z',params=z)

In [ ]:
#plotter(Muv,LF,xlog=False,ylog=True,quant_name='lf',par_name='z',params=z,add_hst=True)
plotter(Muv,LF,xlog=False,ylog=True,quant_name='lf',par_name='z',params=z,add_jwst=True)

Now try increasing the star formation efficiency, $f_{\star}$. Increasing the SFE you might get a better match with the *JWST* data at $z=11$.

In [ ]:
#Given the limiting apparent magnitude (1st arg), area in degree (2nd arg) and the redshift (3rd arg).
#SFE (4th arg, optional)
N = num_gal(30,1,10)

In [ ]:
#Now investigate the number of galaxies for a given survey with redshift.
zs = np.linspace(6,20)
ng = num_gal(30,1,zs)
plotter(zs,ng,xlog=False,ylog=True,quant_name='n',par_name='fs',params=0.1)